In [73]:
import pandas as pd
import numpy as np
from nltk import *

In [291]:
df = pd.read_csv('good_reads_final.csv')
df['book_title'] =df['book_title']
df.head()

,author_average_rating,author_gender,author_genres,author_id,author_name,author_page_url,author_rating_count,author_review_count,birthplace,book_average_rating,book_fullurl,book_id,book_title,genre_1,genre_2,num_ratings,num_reviews,pages,publish_date,score
0,4.010,female,"historical-fiction,",74489,Victoria Thompson\n,/author/show/74489.Victoria_Thompson,74399,6268,United States\n,4.020,https://www.goodreads.com/book/show/686717.Mur...,686717,\n Murder on St. Mark's Place\n,Mystery,Historical,5260,375,277,2000,3230
1,4.150,male,"literature-fiction,mystery-thrillers,",706255,Stieg Larsson\n,/author/show/706255.Stieg_Larsson,3726435,142704,Sweden\n,4.130,https://www.goodreads.com/book/show/2429135.Th...,2429135,\n The Girl with the Dragon Tattoo\n,Fiction,Mystery,2229163,65227,465,August 2005,3062
2,4.000,female,"romance,",5618190,Mimi Jean Pamfiloff\n,/author/show/5618190.Mimi_Jean_Pamfiloff,76496,7975,United States\n,3.990,https://www.goodreads.com/book/show/27833684-t...,27833684,\n Tailored for Trouble\n,Romance,Contemporary,2151,391,354,2016,4585
3,3.880,male,"fiction,memoir,",37871,José Donoso\n,/author/show/37871.Jos_Donoso,5522,489,Chile\n,4.140,https://www.goodreads.com/book/show/382975.The...,382975,\n The Obscene Bird of Night\n,Fiction,Magical Realism,1844,173,438,1970,1533
4,4.100,female,"young-adult,fantasy,",36122,Patricia C. Wrede\n,/author/show/36122.Patricia_C_Wrede,291013,13453,United States\n,4.010,https://www.goodreads.com/book/show/64207.Sorc...,64207,\n Sorcery & Cecelia: or The Enchanted Ch...,Fantasy,Young Adult,17051,1890,326,April 15th 1988,2105


In [292]:
df[df['book_title']== 'TheAlchemist']

,author_average_rating,author_gender,author_genres,author_id,author_name,author_page_url,author_rating_count,author_review_count,birthplace,book_average_rating,book_fullurl,book_id,book_title,genre_1,genre_2,num_ratings,num_reviews,pages,publish_date,score


In [293]:
df.columns

Index(['author_average_rating', 'author_gender', 'author_genres', 'author_id',
       'author_name', 'author_page_url', 'author_rating_count',
       'author_review_count', 'birthplace', 'book_average_rating',
       'book_fullurl', 'book_id', 'book_title', 'genre_1', 'genre_2',
       'num_ratings', 'num_reviews', 'pages', 'publish_date', 'score'],
      dtype='object')

In [294]:
df = df[[ 'book_title', 'author_name', 'book_average_rating','num_ratings','author_average_rating',
         'author_rating_count' , 'genre_1', 'genre_2',  'publish_date', 'book_fullurl']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22891 entries, 0 to 22890
Data columns (total 10 columns):
book_title               22891 non-null object
author_name              22891 non-null object
book_average_rating      22891 non-null float64
num_ratings              22891 non-null int64
author_average_rating    22891 non-null float64
author_rating_count      22891 non-null int64
genre_1                  22891 non-null object
genre_2                  22891 non-null object
publish_date             22610 non-null object
book_fullurl             22891 non-null object
dtypes: float64(2), int64(2), object(6)
memory usage: 1.7+ MB


In [295]:
# Removing the rows with null items in publish date
df.dropna(subset=['publish_date'], inplace=True)
df.rename(columns={"publish_date": "publish_yr"}, inplace=True)

In [296]:
years = []
for dates in df['publish_yr'].str.split(' '):
    try:
        if len(dates) == 1:
            dates = int(dates[0])
        elif len(dates) == 2:
            dates = int(dates[1])
        elif len(dates) == 3:
            dates = int(dates[2])
        
            
    except Exception as e:
        dates = 0
        
        
    years.append(dates)
df['publish_yr'] = years

In [297]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22610 entries, 0 to 22890
Data columns (total 10 columns):
book_title               22610 non-null object
author_name              22610 non-null object
book_average_rating      22610 non-null float64
num_ratings              22610 non-null int64
author_average_rating    22610 non-null float64
author_rating_count      22610 non-null int64
genre_1                  22610 non-null object
genre_2                  22610 non-null object
publish_yr               22610 non-null int64
book_fullurl             22610 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 1.9+ MB


In [298]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.describe()

,book_average_rating,num_ratings,author_average_rating,author_rating_count,publish_yr
count,22610.000,22610.000,22610.000,22610.000,22610.000
mean,3.951,47094.957,3.961,173387.508,1949.534
std,0.289,181063.348,0.239,658157.904,311.349
min,1.850,1.000,1.820,6.000,-720.000
25%,3.770,849.000,3.810,4453.500,1998.000
50%,3.960,4491.500,3.970,25230.500,2011.000
75%,4.140,20451.500,4.120,112362.250,2013.000
max,5.000,3820921.000,5.000,21117318.000,2019.000


In [299]:
df = df[df['publish_yr'] >1980][df['author_rating_count'] > 4000][df['num_ratings']> 800]
df = df.drop(labels=['author_rating_count','num_ratings', 'publish_yr'], axis=1)

c:\users\sahaj oberoi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [300]:
df.head()

,book_title,author_name,book_average_rating,author_average_rating,genre_1,genre_2,book_fullurl
0,\n Murder on St. Mark's Place\n,Victoria Thompson\n,4.020,4.010,Mystery,Historical,https://www.goodreads.com/book/show/686717.Mur...
1,\n The Girl with the Dragon Tattoo\n,Stieg Larsson\n,4.130,4.150,Fiction,Mystery,https://www.goodreads.com/book/show/2429135.Th...
2,\n Tailored for Trouble\n,Mimi Jean Pamfiloff\n,3.990,4.000,Romance,Contemporary,https://www.goodreads.com/book/show/27833684-t...
4,\n Sorcery & Cecelia: or The Enchanted Ch...,Patricia C. Wrede\n,4.010,4.100,Fantasy,Young Adult,https://www.goodreads.com/book/show/64207.Sorc...
5,"\n 30 Days of Night, Vol. 1\n",Steve Niles\n,3.800,3.770,Sequential Art,Sequential Art,https://www.goodreads.com/book/show/831829.30_...


In [301]:
#df['book_title'] = df['book_title'].str.strip() 
def clean_text(df, cols):
    for col in cols:
        text =  []
        df[col] = df[col].str.strip()
        for item in df[col]:
            word = item.replace(" ", "")
            #print(word)
            text.append(word)
        df[col] = text
clean_text(df , ['book_title', 'author_name', 'genre_1', 'genre_2']) 
df.head()

,book_title,author_name,book_average_rating,author_average_rating,genre_1,genre_2,book_fullurl
0,MurderonSt.Mark'sPlace,VictoriaThompson,4.020,4.010,Mystery,Historical,https://www.goodreads.com/book/show/686717.Mur...
1,TheGirlwiththeDragonTattoo,StiegLarsson,4.130,4.150,Fiction,Mystery,https://www.goodreads.com/book/show/2429135.Th...
2,TailoredforTrouble,MimiJeanPamfiloff,3.990,4.000,Romance,Contemporary,https://www.goodreads.com/book/show/27833684-t...
4,Sorcery&Cecelia:orTheEnchantedChocolatePot,PatriciaC.Wrede,4.010,4.100,Fantasy,YoungAdult,https://www.goodreads.com/book/show/64207.Sorc...
5,"30DaysofNight,Vol.1",SteveNiles,3.800,3.770,SequentialArt,SequentialArt,https://www.goodreads.com/book/show/831829.30_...


In [302]:
def numbers_normalised(df, col_list):
    for col in col_list:
        maxx = df[col].max()
        minn = df[col].min()
        df[col] = round((df[col] - minn) / (maxx - minn), 3) 
    return df.head()

numbers_normalised(df, ['book_average_rating', 'author_average_rating'])

,book_title,author_name,book_average_rating,author_average_rating,genre_1,genre_2,book_fullurl
0,MurderonSt.Mark'sPlace,VictoriaThompson,0.649,0.621,Mystery,Historical,https://www.goodreads.com/book/show/686717.Mur...
1,TheGirlwiththeDragonTattoo,StiegLarsson,0.697,0.701,Fiction,Mystery,https://www.goodreads.com/book/show/2429135.Th...
2,TailoredforTrouble,MimiJeanPamfiloff,0.636,0.616,Romance,Contemporary,https://www.goodreads.com/book/show/27833684-t...
4,Sorcery&Cecelia:orTheEnchantedChocolatePot,PatriciaC.Wrede,0.645,0.672,Fantasy,YoungAdult,https://www.goodreads.com/book/show/64207.Sorc...
5,"30DaysofNight,Vol.1",SteveNiles,0.553,0.486,SequentialArt,SequentialArt,https://www.goodreads.com/book/show/831829.30_...


In [305]:
df = df[~df.duplicated()]

In [326]:
df['text'] = df['author_name'] + " " + df['genre_1'] + " " + df['genre_2'] + " " + df['book_title']
df['numerical'] = (1.5 * df['book_average_rating']) + df['author_average_rating']
final_df = df[['book_title', 'text', 'numerical']]
final_df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
final_df.head()

,book_title,text,numerical
0,MurderonSt.Mark'sPlace,VictoriaThompson Mystery Historical MurderonSt...,1.595
1,TheGirlwiththeDragonTattoo,StiegLarsson Fiction Mystery TheGirlwiththeDra...,1.746
2,TailoredforTrouble,MimiJeanPamfiloff Romance Contemporary Tailore...,1.570
3,Sorcery&Cecelia:orTheEnchantedChocolatePot,PatriciaC.Wrede Fantasy YoungAdult Sorcery&Cec...,1.639
4,"30DaysofNight,Vol.1",SteveNiles SequentialArt SequentialArt 30Dayso...,1.316


In [327]:
final_df['numerical'].sort_values(ascending=False).head()

3790   2.494
2454   2.428
4503   2.414
1147   2.406
2832   2.391
Name: numerical, dtype: float64

In [328]:
final_df.tail()

,book_title,text,numerical
9051,DarkWolfRising,RhyannonByrd Fantasy Romance DarkWolfRising,1.462
9052,HiddenMoon,K.R.Thompson YoungAdult Fantasy HiddenMoon,1.503
9053,GiventothePack,AbbyWeeks Fantasy Paranormal GiventothePack,0.982
9054,TheMating:TheOriginalLawoftheLycansStory,NickyCharles Fantasy Romance TheMating:TheOrig...,1.649
9055,MyVikingVampire,KrystalShannan Fantasy Paranormal MyVikingVampire,1.399


In [329]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [330]:
cvv = CountVectorizer()
text_vectorised = cvv.fit_transform(final_df['text']).toarray()

In [331]:
cos_sim = cosine_similarity(text_vectorised, text_vectorised)

In [335]:
book = 'HarryPotterSeriesBoxSet'
idx = final_df[final_df['book_title'] == book ].index
idx
text_10 = pd.Series(cos_sim[idx][0]).sort_values(ascending=False)[1:11]
text_idx = text_10.index
text_recom_idx = (text_10 + final_df['numerical'].loc[text_idx]).sort_values(ascending=False).index
final_df.loc[text_recom_idx]

,book_title,text,numerical
1948,HarryPotterandthePrisonerofAzkaban,J.K.Rowling Fantasy YoungAdult HarryPotterandt...,2.193
5137,HarryPotterandtheOrderofthePhoenix,J.K.Rowling Fantasy YoungAdult HarryPotterandt...,2.147
6163,HarryPotterandtheHalf-BloodPrince,J.K.Rowling Fantasy YoungAdult HarryPotterandt...,2.199
3505,AMonsterCalls,PatrickNess Fantasy YoungAdult AMonsterCalls,1.893
2915,OldMagic,MarianneCurley Fantasy YoungAdult OldMagic,1.536
4821,ForbiddenMind,KarpovKinrade Fantasy YoungAdult ForbiddenMind,1.461
4466,BegforMercy,ShannonDermott Fantasy YoungAdult BegforMercy,1.298
6461,PrincessBen,CatherineGilbertMurdock Fantasy YoungAdult Pri...,1.281
2929,Aurelia,AnneOsterlund YoungAdult Fantasy Aurelia,1.180
2532,Updraft,FranWilde Fantasy YoungAdult Updraft,1.085
